In [78]:
import pandas as pd
import numpy as np
from sklearn import linear_model
import math
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.models import load_model
from keras.layers import LSTM, Dense, Dropout
from sklearn.preprocessing import MinMaxScaler
from keras.losses import CategoricalCrossentropy
from keras.callbacks import EarlyStopping
import tensorflow


In [79]:
#Read data from csv
df = pd.read_csv("cosas_finales\casos_all_3_noN3.txt", error_bad_lines=False)
#df = df.apply(pd.to_numeric)
df

C:\Users\migue\AppData\Local\Temp\ipykernel_12824\362587297.py:2: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  df = pd.read_csv("cosas_finales\casos_all_3_noN3.txt", error_bad_lines=False)


,tamano,tiempo,tamano.1,tiempo.1,tamano.2,tiempo.2,tipo
0,5734,0.000600,28673,0.002500,57347,0.001400,0
1,66349,0.001000,331748,0.000100,663496,0.000100,0
2,50538,0.000300,252693,0.000700,505387,0.002100,0
3,25233,0.000200,126167,0.001700,252334,0.002000,0
4,59710,0.001800,298553,0.001800,597106,0.001700,0
...,...,...,...,...,...,...,...
6999995,748,0.000157,3742,0.003921,7484,0.007841,4
6999996,330,0.000381,1650,0.004574,3300,0.019820,4
6999997,1327,0.004191,6635,0.061633,13271,0.073172,4
6999998,1545,0.001003,7725,0.100255,15450,0.047657,4


In [80]:
for i in range(0, len(df)-1):
  #print(str(df.tipo.values[i]) + " " + str(i))
  if df.tipo.values[i] < 0 or df.tipo.values[i] > 5:
    df = df.drop(i, axis=0)

df = df.dropna()
df

,tamano,tiempo,tamano.1,tiempo.1,tamano.2,tiempo.2,tipo
0,5734,0.000600,28673,0.002500,57347,0.001400,0
1,66349,0.001000,331748,0.000100,663496,0.000100,0
2,50538,0.000300,252693,0.000700,505387,0.002100,0
3,25233,0.000200,126167,0.001700,252334,0.002000,0
4,59710,0.001800,298553,0.001800,597106,0.001700,0
...,...,...,...,...,...,...,...
6999995,748,0.000157,3742,0.003921,7484,0.007841,4
6999996,330,0.000381,1650,0.004574,3300,0.019820,4
6999997,1327,0.004191,6635,0.061633,13271,0.073172,4
6999998,1545,0.001003,7725,0.100255,15450,0.047657,4


In [81]:
#drop type column to obtain the x_train data use 6 if size included 3 if not
x_train = df.iloc[:, 0:6]
#x_train = df.iloc[:, 0:3]

#y_train = df.iloc[:,6]
x_train

,tamano,tiempo,tamano.1,tiempo.1,tamano.2,tiempo.2
0,5734,0.000600,28673,0.002500,57347,0.001400
1,66349,0.001000,331748,0.000100,663496,0.000100
2,50538,0.000300,252693,0.000700,505387,0.002100
3,25233,0.000200,126167,0.001700,252334,0.002000
4,59710,0.001800,298553,0.001800,597106,0.001700
...,...,...,...,...,...,...
6999995,748,0.000157,3742,0.003921,7484,0.007841
6999996,330,0.000381,1650,0.004574,3300,0.019820
6999997,1327,0.004191,6635,0.061633,13271,0.073172
6999998,1545,0.001003,7725,0.100255,15450,0.047657


In [82]:
y_train = []
for i in range(0,len(df)):
  high = df.tipo.values[i]
  if high == 0:
    y_train.append([1,0,0,0,0])
  elif high == 1:
    y_train.append([0,1,0,0,0])
  elif high == 2:
    y_train.append([0,0,1,0,0])
  elif high == 3:
    y_train.append([0,0,0,1,0])
  elif high == 4:
    y_train.append([0,0,0,0,1])
  #elif high == 5:
  #  y_train.append([0,0,0,0,0,1])
  else:
#    x_train.delete(i)
    print(str(i) + " " + str(df.tipo.values[i]))

len(y_train)

7000000

In [83]:
#reshape data to the type the AI model needs
x_train, y_train = np.array(x_train), np.array(y_train)
x_train = np.asarray(x_train).astype('float32')
y_train = np.asarray(y_train).astype('float32')

In [84]:
real_df = pd.read_csv("cosas_finales/tiempos.csv", error_bad_lines=False)
x_train_val = real_df.iloc[:, 1:7]

y_train_val = []
for i in range(0,len(real_df)):
  high = real_df.tipo.values[i]
  if high == 0:
    y_train_val.append([1,0,0,0,0])
  elif high == 1:
    y_train_val.append([0,1,0,0,0])
  elif high == 2:
    y_train_val.append([0,0,1,0,0])
  elif high == 3:
    y_train_val.append([0,0,0,1,0])
  elif high == 4:
    y_train_val.append([0,0,0,0,1])
  #elif high == 5:
  #  y_train_val.append([0,0,0,0,0,1])
  else:
    print(str(i) + " " + str(real_df.tipo.values[i]))

x_train_val = np.array(x_train_val)
y_train_val = np.array(y_train_val)

C:\Users\migue\AppData\Local\Temp\ipykernel_12824\3164716819.py:1: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  real_df = pd.read_csv("cosas_finales/tiempos.csv", error_bad_lines=False)


Note, the most confusing thing here is that the shape of the input to the model is defined as an argument on the first hidden layer. This means that the line of code that adds the first layer is doing 2 things, defining the input or visible layer and the first hidden layer.

In [102]:
#Generate a sequential model with 4 layers 
#The last one has 6 nodes, one for each execution time order [cte, logn, n, nlogn, n2, n3]
#The ones in between have arbitrary values obtain through testing ( still may be swapped )

#redes GRU igual esta interesante

model = Sequential()
#model.add(no. de entradas de cada caso) 
#esto es asi, en la libreria keras, en el modelo secuencial, se define en input_shape el nº de nodos de la entrada
model.add(LSTM(100, return_sequences=True, input_shape=(x_train.shape[1], 1)))
model.add(Dropout(0.2))
model.add(LSTM(100, return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(100, return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(100, return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(100, return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(100))
model.add(Dropout(0.2))
#model.add(Dense(50))
model.add(Dense(5, activation = 'softmax'))

#model.add(Dense(50, activation='relu'))
#model.add(Dense(50, activation='relu'))
#model.add(Dense(25, activation='relu'))
#model.add(Dense(5, activation='relu'))

#Compile the model previously created with the chosen parameters
#optimizer='sgd'
model.compile(optimizer='adam', loss = CategoricalCrossentropy(), metrics=['accuracy'])
model.summary()

Model: "sequential_16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_84 (LSTM)              (None, 6, 100)            40800     
                                                                 
 dropout_84 (Dropout)        (None, 6, 100)            0         
                                                                 
 lstm_85 (LSTM)              (None, 6, 100)            80400     
                                                                 
 dropout_85 (Dropout)        (None, 6, 100)            0         
                                                                 
 lstm_86 (LSTM)              (None, 6, 100)            80400     
                                                                 
 dropout_86 (Dropout)        (None, 6, 100)            0         
                                                                 
 lstm_87 (LSTM)              (None, 6, 100)          

In [103]:
#Train the model with the 'train' portion of the data 
#The other parameters are tuned down in order to reduce train time but should be higher
#callback = EarlyStopping(monitor='val_accuracy', patience = 5, restore_best_weights=True)
#model.fit(x_train, y_train, batch_size=2048, epochs=10, callbacks=[callback], validation_data=(x_train_val, y_train_val))

model.fit(x_train, y_train, batch_size=1024, epochs=10, validation_data=(x_train_val, y_train_val))


Epoch 1/10
6836/6836 [==============================] - 2882s 420ms/step - loss: 0.9053 - accuracy: 0.6099 - val_loss: 4.0123 - val_accuracy: 0.1429
Epoch 2/10
6836/6836 [==============================] - 3001s 439ms/step - loss: 0.6710 - accuracy: 0.7264 - val_loss: 5.1446 - val_accuracy: 0.1429
Epoch 3/10
6836/6836 [==============================] - 3022s 442ms/step - loss: 0.5374 - accuracy: 0.7858 - val_loss: 4.9530 - val_accuracy: 0.1429
Epoch 4/10
6836/6836 [==============================] - 3069s 449ms/step - loss: 0.4816 - accuracy: 0.8109 - val_loss: 5.7818 - val_accuracy: 0.2143
Epoch 5/10
6836/6836 [==============================] - 3123s 457ms/step - loss: 0.5029 - accuracy: 0.8008 - val_loss: 4.1866 - val_accuracy: 0.1429
Epoch 6/10
6836/6836 [==============================] - 3186s 466ms/step - loss: 0.4809 - accuracy: 0.8102 - val_loss: 5.3034 - val_accuracy: 0.1429
Epoch 7/10
6836/6836 [==============================] - 3270s 478ms/step - loss: 0.4010 - accuracy: 0.8447

In [104]:
#Save trained model to a file
model.save("model.h5")

In [105]:
#Use the model to predict the test data in order to find the success percent in the test data

predictions = model.predict(x_train_val)
predictions

array([[5.53763530e-04, 1.46638602e-01, 6.79203481e-07, 7.88657546e-01,
        6.41494542e-02],
       [4.20105636e-01, 1.61323190e-01, 9.44070593e-02, 3.24093193e-01,
        7.08802181e-05],
       [9.75783542e-02, 1.09781824e-01, 9.13602416e-05, 7.92349398e-01,
        1.99058544e-04],
       [2.97576701e-08, 9.99495029e-01, 3.99604652e-08, 5.04806696e-04,
        8.53562554e-09],
       [2.47185852e-07, 1.30862862e-01, 1.20554557e-06, 8.69105756e-01,
        2.99076655e-05],
       [2.38956526e-01, 7.58625031e-01, 4.46197009e-06, 2.40249079e-04,
        2.17372621e-03],
       [2.77830935e-07, 1.53540736e-02, 2.24237540e-09, 2.02347466e-04,
        9.84443307e-01],
       [7.86128112e-06, 9.99908447e-01, 4.45635262e-09, 8.03732473e-05,
        3.34486685e-06],
       [4.34646994e-01, 2.60434568e-01, 6.61203172e-04, 3.04221660e-01,
        3.55115444e-05],
       [2.06934997e-10, 1.00000000e+00, 8.04300029e-12, 2.13261870e-08,
        3.04850678e-09],
       [9.92919028e-01, 8.9730

In [106]:
real_preds = []
for i in range(0,len(predictions)):
  p = predictions[i]
  if max(p) == p[0]:
    real_preds.append([1,0,0,0,0])
  elif max(p) == p[1]:
    real_preds.append([0,1,0,0,0])
  elif max(p) == p[2]:
    real_preds.append([0,0,1,0,0])
  elif max(p) == p[3]:
    real_preds.append([0,0,0,1,0])
  elif max(p) == p[4]:
    real_preds.append([0,0,0,0,1])
  #elif max(p) == p[5]:
  #  real_preds.append([0,0,0,0,0,1])

#Calculate error generated in the "real"predictions of test data
err = real_preds - y_train_val
err

array([[ 0,  0, -1,  1,  0],
       [ 1,  0, -1,  0,  0],
       [ 0,  0, -1,  1,  0],
       [ 0,  1,  0, -1,  0],
       [ 0,  0,  0,  0,  0],
       [ 0,  1,  0,  0, -1],
       [ 0,  0,  0,  0,  0],
       [ 0,  1,  0,  0, -1],
       [ 1,  0, -1,  0,  0],
       [-1,  1,  0,  0,  0],
       [ 1, -1,  0,  0,  0],
       [ 1, -1,  0,  0,  0],
       [ 0,  0,  0,  0,  0],
       [ 1, -1,  0,  0,  0]])

In [107]:
def numToType( t):
  if t==0: return "CTE"
  elif t==1:return "LOG"
  elif t==2:return "N"
  elif t==3:return "NLOGN"
  elif t==4:return "N^2"
  elif t==5:return "N^3"

error_count = 0
for i in range(0, len(err)):
  if max(err[i]) != min(err[i]):
    print(str(real_df.name.values[i]) + " sol: " + numToType(real_df.tipo.values[i]) + "  pred: " + numToType(np.argmax(real_preds[i])))
    error_count += 1
print(str(error_count) + " / " + str(len(real_preds)) + " => real error% of " + str(error_count/len(real_preds)*100))

niniosPrimero sol: N  pred: NLOGN
palmerasNieve sol: N  pred: CTE
--erasmus sol: N  pred: NLOGN
mergeSort sol: NLOGN  pred: LOG
--cazaTesoros sol: N^2  pred: LOG
tiroPalindromo sol: N^2  pred: LOG
--palmerasNieve2invent sol: N  pred: CTE
-114 sol: CTE  pred: LOG
-cajasBolas sol: LOG  pred: CTE
-167fractales sol: LOG  pred: CTE
--323 sol: LOG  pred: CTE
11 / 14 => real error% of 78.57142857142857


In [108]:
expected_train = []
for i in range(0,len(real_df)):
  high = real_df.tipoProblema.values[i]
  if high == 0:
    expected_train.append([1,0,0,0,0])
  elif high == 1:
    expected_train.append([0,1,0,0,0])
  elif high == 2:
    expected_train.append([0,0,1,0,0])
  elif high == 3:
    expected_train.append([0,0,0,1,0])
  elif high == 4:
    expected_train.append([0,0,0,0,1])
  #elif high == 5:
  #  expected_train.append([0,0,0,0,0,1])
  else:
    print(str(i) + " " + str(real_df.tipo.values[i]))


AttributeError: 'DataFrame' object has no attribute 'tipoProblema'

In [ ]:
#error teniendo en cuenta la aplicacion real del programa ( no todos los fallos son fallos )
#ejemplo: espero un lineal y entregan un n2, lo interpreto como n3, el resultado seria "no es lineal" y no seria un fallo
#creo que este tipo de error_count solo se puede calcular aplicando el modelo a un problema donde espero un resultado y entregan otro, no se como hacerlo aqui

#y_train = []            #tipo de ejecucion real del programa
#expected_train = []     #tipo de ejecucion esperado por el problema
predictions_train = real_preds  #ordenes predecidos por la ia

correct = 0
partial_corrects = 0
errors = 0
for i in range(0, len(real_preds)):
  guess_id = np.argmax(predictions_train[i])
  answer_id = np.argmax(y_train[i])
  expected_id = np.argmax(expected_train[i])
  if guess_id == answer_id:
    correct += 1
  elif guess_id > answer_id and expected_id != answer_id:
    partial_corrects += 1
  else:
    errors += 1

print(str(errors) + " / " + str(len(real_preds)) + " => useful error% of " + str(errors/len(real_preds)*100))
print(str(correct) + " + " + str(partial_corrects) + " = " + str(correct+partial_corrects)  + " / " + str(len(real_preds)) + " => useful guesses% of " + str((correct+partial_corrects)/len(real_preds)*100))

#error_count = 0
#for i in range(0, len(real_preds)):
#  guess_id = np.argmax(real_preds[i])
#  answer_id = np.argmax(y_test[i])
#  if guess_id < answer_id:
#    error_count += 1
#print(str(error_count) + " / " + str(len(real_preds)) + " => useful error% of " + str(error_count/len(real_preds)*100))

IndexError: list index out of range